---
title: IDs from Juno
order: 10
---

See following notebooks for details:

- [Magnetic field data](./mag.ipynb)
- [State data](./state.ipynb)

## Background

Spacecraft-Solar equatorial

https://pds-ppi.igpp.ucla.edu/data/JNO-SS-3-FGM-CAL-V1.0/INDEX/INDEX.TAB


```txt
------------------------------------------------------------------------------
Juno Mission Phases                                                           
------------------------------------------------------------------------------
Start       Mission                                                           
Date        Phase                                                             
==============================================================================
2011-08-05  Launch                                                            
2011-08-08  Inner Cruise 1                                                    
2011-10-10  Inner Cruise 2                                                    
2013-05-28  Inner Cruise 3                                                    
2013-11-05  Quiet Cruise                                                      
2016-01-05  Jupiter Approach                                                  
2016-06-30  Jupiter Orbital Insertion                                         
2016-07-05  Capture Orbit                                                     
2016-10-19  Period Reduction Maneuver                                         
2016-10-20  Orbits 1-2                                                        
2016-11-09  Science Orbits                                                    
2017-10-11  Deorbit
```

### Coordinate System of Data

1. **SE (Solar Equatorial)**
    - Code: `se`
    - Resampling options: 
        - Number of seconds (1 or 60): `se_rN[N]s`
        - Resampled 1 hour: `se_r1h`

2. **PC (Planetocentric)**
    - Code: `pc`
    - Resampling options: 
        - Number of seconds (1 or 60): `pc_rN[N]s`
        
3. **SS (Sun-State)**
    - Code: `ss`
    - Resampling options: 
        - Number of seconds (1 or 60): `ss_rN[N]s`
        
4. **PL (Payload)**
    - Code: `pl`
    - Resampling options: 
        - Number of seconds (1 or 60): `pl_rN[N]s`

There are three principal coordinate systems used to represent the data in this archive.

- The SE coordinate system is a Spacecraft- Solar equatorial system and it will be used for cruise data only. 
- The sun-state (ss) and planetocentric (pc) will be used for Earth Fly By (EFB) and Jupiter orbital data. 
- Cartesian representations are used for all three coordinate systems. These coordinate systems are specified relative to a “target body” which may be any solar system object (but for this orbital operations will Jupiter). In what follows we will reference Jupiter as the target body, but, for example, if observations near a satellite (such as Io) are desired in Io-centric coordinates, the satellite Io may be specified as the target body. 

The SE coordinate system is defined using the sun-spacecraft vector as the primary reference vector; sun’s rotation axis as the secondary reference vector (z). The x axis lies along the sun-spacecraft vector, the z axis is in the plane defined by the Sun’s rotation axis and the spacecraft-sun vector. The y axis completes the system.

The ss coordinate system is defined using the instantaneous Jupiter-Sun vector as the primary reference vector (x direction). The X-axis lies along this vector and is taken to be positive toward the Sun. The Jupiter orbital velocity vector is the second vector used to define the coordinate system; the y axis lies in the plane determined by the Jupiter-Sun vector and the velocity vector and is orthogonal to the x axis (very nearly the negative of the velocity vector). The vector cross product of x and y yields a vector z parallel to the northward (upward) normal of the orbit plane of Jupiter. This system is sometimes called a sun-state (ss) coordinate system since its principal vectors are the Sun vector and the Jupiter state vector.

## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create juno
```

In [1]:
from discontinuitypy.datasets import IDsDataset
import polars as pl
from fastcore.utils import walk

07-Feb-24 13:36:36: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/__init__.py", line 85, in <module>
    from . import skintegrate
  File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/skintegrate.py", line 20, in <module>
    from sklearn.base import BaseEstimator
ModuleNotFoundError: No module named 'sklearn'


07-Feb-24 13:36:36: UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.

07-Feb-24 13:36:36: UserWarning: Traceback (most recent call last):
  File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/__init__.py", line 105, in <module>
    from . import nltk_stages
  File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/nltk_stages.py", line 19, in <module>
    import nltk
ModuleNotFoundError: No module named 'nltk'


07-Feb-24 13:36:36: UserWarning: pdpipe: nltk import

In [8]:
ts = 1 # unit: seconds
tau = 60 # unit: seconds

data_dir = '../../../data/'
dir_path = f'{data_dir}/03_primary/JNO_MAG_ts_{ts}s'
juno_state_path = f'{data_dir}/03_primary/JNO_STATE_ts_3600s.parquet'
vec_cols = ['v_x', 'v_y', 'v_z']

format = 'arrow'
fname = f'events.JNO.ts_{ts}s_tau_{tau}s.{format}'
output_path = f'{data_dir}/05_reporting/{fname}'

In [3]:
juno_events = []
for mag_path in walk(dir_path):

    juno_mag_data = pl.scan_parquet(mag_path).drop('X', 'Y', 'Z').sort('time')
    juno_plasma_data = pl.scan_parquet(juno_state_path).sort('time')

    _juno_events = (
        IDsDataset(
            mag_data=juno_mag_data,
            plasma_data=juno_plasma_data,
            tau=tau,
            ts=ts,
            vec_cols=vec_cols,
        )
        .find_events(return_best_fit=False)
        .update_candidates_with_plasma_data()
        .events
    )
    
    juno_events.append(_juno_events)
    
juno_ids_dataset = IDsDataset(
    events=pl.concat(juno_events),
    mag_data= pl.scan_parquet(list(walk(dir_path))).drop('X', 'Y', 'Z').sort('time')
).export(output_path)

07-Feb-24 13:36:41: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.

07-Feb-24 13:36:42: UserWarning: Ray execution environment not yet initialized. Initializing...
To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()


2024-02-07 13:36:44,211	INFO worker.py:1724 -- Started a local Ray instance.
07-Feb-24 13:36:45: UserWarning: Distributing <class 'pandas.core.frame.DataFrame'> object. This may take some time.



Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

07-Feb-24 13:36:45: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False
(_deploy_ray_func pid=65621) UserWarning: Traceback (most recent call last):
(_deploy_ray_func pid=65621)   File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/__init__.py", line 85, in <module>
(_deploy_ray_func pid=65621)     from . import skintegrate
(_deploy_ray_func pid=65621)   File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/skintegrate.py", line 20, in <module>
(_deploy_ray_func pid=65621)     from sklearn.base import BaseEstimator
(_deploy_ray_func pid=65621) ModuleNotFoundError: No module named 'sklearn'
(_deploy_ray_func pid=65621) 
(_deploy_ray_func pid=65621) UserWarning: pdpipe: Scikit-learn or skutil import failed. Scikit-learn-dependent pipeline stages will not be loaded.
(_deploy_ray_func pid=65621) UserWarning: Traceback (most recent call la

Estimated completion of line 8:   0%           Elapsed time: 00:00, estimated remaining time: ?

(_deploy_ray_func pid=65621) RuntimeWarning: overflow encountered in exp
(_deploy_ray_func pid=65622) UserWarning: Traceback (most recent call last): [repeated 22x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_deploy_ray_func pid=65622)   File "/Users/zijin/micromamba/envs/juno/lib/python3.10/site-packages/pdpipe/nltk_stages.py", line 19, in <module> [repeated 44x across cluster]
(_deploy_ray_func pid=65622)     from . import skintegrate [repeated 11x across cluster]
(_deploy_ray_func pid=65622)     from sklearn.base import BaseEstimator [repeated 11x across cluster]
(_deploy_ray_func pid=65622) ModuleNotFoundError: No module named 'sklearn' [repeated 11x across cluster]
(_deploy_ray_func pid=65620)  [repeated 29x across cluster]
(_deploy_ray_func pid=65622) UserWarning: pdpipe: Scikit-learn or skutil import failed. Sciki

...

07-Feb-24 13:38:52: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

07-Feb-24 13:38:52: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=True; async_mode_on=False


Distributing Dataframe:   0%           Elapsed time: 00:00, estimated remaining time: ?

07-Feb-24 13:38:52: Using sequential splitting in '.from_pandas()' because of some of the conditions are False: enough_elements=False; all_numeric_types=False; async_mode_on=False


## Processing the whole data

## Obsolete

### Estimate

1 day of data resampled by 1 sec is about 12 MB.

So 1 year of data is about 4 GB, and 6 years of JUNO Cruise data is about 24 GB.

Downloading rate is about 250 KB/s, so it will take about 3 days to download all the data.

In [ ]:
num_of_files = 6*365
jno_file_size = 12e3
thm_file_size = 40e3
files_size = jno_file_size + thm_file_size
downloading_rate = 250
processing_rate = 1/60

time_to_download = num_of_files * files_size / downloading_rate / 60 / 60
space_required = num_of_files * files_size / 1e6
time_to_process = num_of_files / processing_rate / 60 / 60

print(f"Time to download: {time_to_download:.2f} hours")
print(f"Disk space required: {space_required:.2f} GB")
print(f"Time to process: {time_to_process:.2f} hours")

Time to download: 126.53 hours
Disk space required: 113.88 GB
Time to process: 36.50 hours
